In [1]:
from gensim.models import FastText
import re
from multiprocessing import Pool
import os
import pandas as pd
from gensim.models.phrases import Phrases, Phraser

In [2]:
from functools import lru_cache

In [3]:
greystone_notes = pd.read_parquet('/data/greystone_progress_notes_all_time.parquet')
greystone_notes['client'] = 'greystone'

avante_notes = pd.read_parquet('/data/avante_notes_valid_types_only.parquet')

In [4]:
note_columns = set(avante_notes.columns) & set(greystone_notes.columns)
all_notes = greystone_notes[note_columns].append(avante_notes[note_columns]).copy()
all_notes = all_notes.sort_values(['client', 'CreatedDate', 'ProgressNoteID', 'Section', 'SectionSequence'])

In [5]:
grp_cols = ['client', 'FacilityID', 'PatientID', 'CreatedDate', 'ProgressNoteID', 'ProgressNoteType', 'Section']

In [6]:
all_notes_grouped = all_notes.groupby(grp_cols).agg({'NoteText': lambda note_parts: ''.join(note_parts)}).reset_index()

In [7]:
all_notes_grouped.head()

,client,FacilityID,PatientID,CreatedDate,ProgressNoteID,ProgressNoteType,Section,NoteText
0,avante,1,-1.0,2013-12-08 22:00:38.010,3828176,z R.T. Shift Note (7pm - 7am),Ambu Bag / O2 Tank / Spare Trach in place (Yes...,y
1,avante,1,-1.0,2013-12-08 22:00:38.010,3828176,z R.T. Shift Note (7pm - 7am),Narrative Note,Resident received in bed on 28% 02 as ordered ...
2,avante,1,-1.0,2015-01-11 19:51:51.413,8393661,z R.T. Shift Note (7pm - 7am),Ambu Bag / O2 Tank / Spare Trach in place (Yes...,
3,avante,1,-1.0,2015-01-11 19:51:51.413,8393661,z R.T. Shift Note (7pm - 7am),Narrative Note,I made a mistake on RR at 17:07 yesterday char...
4,avante,1,-1.0,2016-03-17 09:50:26.927,13889175,z R.T. Shift Note (7am-7pm),Ambu Bag / O2 Tank / Spare Trach in place (Yes...,y


In [9]:
def preprocess(strings):
    for s in strings:
        yield preprocess_one(s)

@lru_cache(maxsize=500000)
def preprocess_one(s):
    s = s.lower()
    tokens = re.split(r'\s', s)
    return tuple(t for t in tokens if len(t) > 0)

In [ ]:
pool = Pool(os.cpu_count() - 2)

In [11]:
tokens = pool.map(preprocess_one, all_notes_grouped['NoteText'].values)

In [ ]:
phrases = Phrases(tokens, min_count=500)
bigram = Phraser(phrases)
del phrases

In [20]:
del pool

In [21]:
def get_bigrams(sentence_tokens):
    return bigram[sentence_tokens]

def get_trigrams(sentence_tokens):
    return trigram[sentence_tokens]

In [22]:
pool = Pool(os.cpu_count() - 2)

Process ForkPoolWorker-169:
Process ForkPoolWorker-175:
Process ForkPoolWorker-157:
Process ForkPoolWorker-166:
Process ForkPoolWorker-142:
Process ForkPoolWorker-139:
Process ForkPoolWorker-155:
Process ForkPoolWorker-170:
Process ForkPoolWorker-141:
Process ForkPoolWorker-144:
Process ForkPoolWorker-181:
Process ForkPoolWorker-167:
Process ForkPoolWorker-177:
Process ForkPoolWorker-178:
Process ForkPoolWorker-179:
Process ForkPoolWorker-153:
Process ForkPoolWorker-171:
Process ForkPoolWorker-161:
Process ForkPoolWorker-160:
Process ForkPoolWorker-168:
Process ForkPoolWorker-172:
Process ForkPoolWorker-152:
Process ForkPoolWorker-165:
Process ForkPoolWorker-184:
Process ForkPoolWorker-156:
Process ForkPoolWorker-154:
Process ForkPoolWorker-174:
Process ForkPoolWorker-151:
Process ForkPoolWorker-180:
Process ForkPoolWorker-140:
Process ForkPoolWorker-158:
Process ForkPoolWorker-183:
Process ForkPoolWorker-149:
Process ForkPoolWorker-145:
Process ForkPoolWorker-176:
Process ForkPoolWork

  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "

  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with

  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/mul

In [25]:
%%time
_ = bigram[tokens[:1000]]

CPU times: user 302 µs, sys: 16 µs, total: 318 µs
Wall time: 325 µs


In [24]:
%%time
_ = pool.map(get_bigrams, tokens[:1000])

CPU times: user 10 ms, sys: 108 µs, total: 10.1 ms
Wall time: 8.1 ms


In [26]:
%%time
bigram_tokens = bigram[tokens]

CPU times: user 17 µs, sys: 1e+03 ns, total: 18 µs
Wall time: 21.2 µs


In [31]:
%%time
trigram_phrases = Phrases(bigram_tokens, min_count=500)
trigram = Phraser(trigram_phrases)
del trigram_phrases

CPU times: user 38min 44s, sys: 51.9 s, total: 39min 35s
Wall time: 39min 8s


In [32]:
trigram_tokens = trigram[bigram[tokens]]

In [38]:
sample['tokens'] = trigram_tokens[:5]

In [41]:
import numpy as np

In [51]:
import pickle

In [53]:
with open('trigram_tokens.p' ,'wb') as f:
    pickle.dump(trigram_tokens, f)

## Read from disk

In [8]:
import pickle

In [9]:
with open('/data/trigram_tokens.p', 'rb') as f:
    trigram_tokens = pickle.load(f)

In [11]:
sample = all_notes_grouped.head(10000).copy()

In [15]:
import numpy as np

In [16]:
%%time
sample['tokens'] = np.asarray(trigram_tokens[:10000])

CPU times: user 3.35 s, sys: 25 ms, total: 3.37 s
Wall time: 3.37 s


In [17]:
%%time
sample['tokens'] = trigram_tokens[:10000]

CPU times: user 641 ms, sys: 11.5 ms, total: 652 ms
Wall time: 651 ms


In [19]:
%%time
token_s = pd.Series(trigram_tokens[:10000])
sample['tokens'] = token_s

CPU times: user 5.95 s, sys: 47.4 ms, total: 6 s
Wall time: 6 s


In [23]:
%%time
np.concatenate([sample, trigram_tokens[:10000]])

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [ ]:
all_notes_grouped['tokens'] = trigram_tokens

In [49]:
bigram.save('/data/models/bigram_phraser.model')
trigram.save('/data/models/trigram_phraser.model')

/opt/conda/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/opt/conda/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
all_notes_grouped.to_pickle('/data/all_notes_tokenized.pickle')

In [ ]:
is_emar = (all_notes_grouped['ProgressNoteType'] == 'eMAR-Medication Administration Note') | (all_notes_grouped['ProgressNoteType'] == 'eMar - General Note from eRecord')

In [ ]:
emar_notes = all_notes_grouped.loc[is_emar]

In [ ]:
other_notes = all_notes_grouped.loc[~is_emar]

In [ ]:
assert len(emar_notes) + len(other_notes) == len(all_notes_grouped)

In [ ]:
emar_notes.to_pickle('/data/emar_notes_tokenized.pickle')
other_notes.to_pickle('/data/non_emar_notes_tokenized.pickle')

In [17]:
len(greystone_notes[greystone_notes['ProgressNoteType'].isin(note_types)])

0

In [6]:
avante_notes = pd.read_parquet('/data/avante_notes_valid_types_only.parquet')

In [1]:
tokens = pool.map(preprocess_one, full_notes[full_notes['NoteText']]['NoteText'])

NameError: name 'pool' is not defined

In [ ]:
def preprocess(strings):
    for s in strings:
        yield preprocess_one(s)

def preprocess_one(s):
    s = s.lower()
    tokens = re.split(r'\s', s)
    return tuple(t for t in tokens if len(t) > 0)